An analysis on creating the joint positions / time for our quad wave gait.

# Highest Imports

Set up matplotlib

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt

Other useful imports

In [2]:
import numpy as np

# Gait Parameters

In [3]:
# beta = duty_cycle = 0.75
# v = body_velocity = 0.03  # m/s
# T = cycle_time = 1  # s
# L = v * T # m

In [4]:
beta = duty_cycle = 0.75
v = body_velocity = 0.32  # m/s
T = cycle_time = .75  # s
L = v * T # m
L

0.24

Compute $T_l$, the relative cycle time for each leg

In [5]:
T_l = (1 - beta) * T

Vertical Leg movement parameters

In [6]:
# Maximum distance to raise the foot tip
y_max = .035  #m

# Generate Horizontal + Vertical Positions & Velocites

We will split this up into 6 checkpoints, similar to how it was done in the video. 
1. vertical launch (when the leg is lifting)
2. horizontal launch (leg is done lifting up and starts moving horizontally)
3. horizontal movement (leg is moving the final position)
4. horizontal movement stop
5. horizontal stop
6. vertical stop

Observe that the horizontal velocity graph is always symmetric, so we can split our checkpoints into the two following groups:
1. horizontal_move_start
2. horizontal_move_stable

As the leg always needs to move first, we know that $launch_{vertical} = 0$ always. Keep them as fractions of the **half cycle** as everything is symmetric and dynamic

In [7]:
horizontal_move_start = .4
horizontal_move_stable = .8

checkpoint_fracs = np.array((0, horizontal_move_start, horizontal_move_stable))
left = checkpoint_fracs * T_l / 2
right = T_l - left 
t = np.concatenate((left, np.flip(right)))
t

array([0.    , 0.0375, 0.075 , 0.1125, 0.15  , 0.1875])

Visualize the checkpoints

In [8]:
plt.figure()
plt.scatter(t, np.full(t.shape, 1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Horizontal Velocity Over Time
Compute the x-velocities for all timesteps

In [9]:
vx_max = L / ((t[2] - t[1]) + (t[3] - t[2]))
vx = np.array((0, 0, vx_max, vx_max, 0, 0))

In [10]:
L

0.24

Visualize

In [11]:
plt.figure()
plt.plot(t, vx)
plt.title('$v_x$ vs $t$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$v_x$ vs $t$')

## Horizontal Displacement over time
Recall that position ($x$) can be found as

$$x(t + \Delta T) = x(t) + \left. \int vx dx \right]_{t-1}^{t}$$

In [12]:
def vel_to_pos(start, t, vels):
    d_curr = start
    d = [d_curr]
    for i in range(1, len(t)):        
        if vels[i] == vels[i - 1]:  # Square integration
            d_curr += (t[i] - t[i - 1]) * vels[i]
        else:
            if vels[i] == 0:
                sgn = np.sign(vels[i - 1])
            else:
                sgn = np.sign(vels[i])
            
            # Compute square area
            square = (t[i] - t[i - 1]) * min(abs(vels[i]), abs(vels[i - 1]))
            
            # Compute triangle area
            triangle = .5 * (t[i] - t[i - 1]) * abs(vels[i] - vels[i - 1]) * sgn
            
            d_curr += (square + triangle)
        d.append(d_curr)
    return np.array(d)

Recall that the position graph can be found for

In [13]:
x_0 = -L / 2
x = vel_to_pos(x_0, t, vx)

In [14]:
t

array([0.    , 0.0375, 0.075 , 0.1125, 0.15  , 0.1875])

Visualize

In [15]:
plt.figure()
plt.plot(t, x)
plt.title('$x$ vs $t$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$x$ vs $t$')

## Vertical Velocity over Time

In [16]:
vy_max = 2 * y_max / (t[2] - t[0]) 
vy = np.array([0, vy_max, 0, 0, -vy_max, 0])

Visualize

In [17]:
plt.figure()
plt.plot(t, vy)
plt.title('$vy$ vs $t$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$vy$ vs $t$')

## Vertical Displacement Over Time

In [18]:
y_0 = 0
y = vel_to_pos(y_0, t, vy)

Visualize

In [19]:
plt.figure()
plt.plot(t, y)
plt.title('$y$ vs $t$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$y$ vs $t$')

## Vertical Vel vs. Horizontal Vel w.r.t Ground

In [20]:
plt.figure()
plt.plot(x, y)
plt.title('$y$ vs $x$ w.r.t ground')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$y$ vs $x$ w.r.t ground')

## Vertical Positiosn vs. Horizontal positions w.r.t Body

In [21]:
x_b = x - v * t

Visualize

In [22]:
plt.figure()
plt.plot(x_b, y)
plt.title('$y$ vs $x$ w.r.t body')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$y$ vs $x$ w.r.t body')

In [23]:
res = []
for i, t_ in enumerate(t):
    res.append((t_, (x_b[i], y[i])))
res

[(0.0, (-0.12, 0.0)),
 (0.037500000000000006, (-0.132, 0.0175)),
 (0.07500000000000001, (-0.08399999999999998, 0.035)),
 (0.11249999999999999, (0.02399999999999998, 0.035)),
 (0.15, (0.072, 0.0175)),
 (0.1875, (0.06, 0.0))]